In [1]:
import tensorflow.keras as keras
import numpy as np
from scipy.io import loadmat
from my_classes import DataGenerator2
import matplotlib.pyplot as plt
import glob
import pickle
%matplotlib inline

model_from_json = keras.models.model_from_json

types = ['kmno4', 'set1', 'set2', 'set5', 'set9']
ModelCheckpoint=keras.callbacks.ModelCheckpoint
Sequential=keras.models.Sequential
Dense=keras.layers.Dense
Flatten=keras.layers.Flatten
Dropout=keras.layers.Dropout
np.random.seed(7)

/home/jayathu/anaconda3/envs/ml_35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


The main purpose of creating a data generator is to avoid unnecessary usage of RAM when using larger data sets. `my_classes.py` contains such a generator.

Creating `partition` dictionary

In [ ]:
# {'train':[['filename', 'position'], ['filename2','position2']]}

# l=[]

# for matfile in glob.glob('unsieved_concat/*.mat'):
#     for i in range(56250-64):
#         l.append([matfile,i])

# np.random.shuffle(l)

# l[:5]

# total_data = len(l)

# train_size = int(total_data * 70 / 100)
# train_size

# val_size = int((total_data - train_size) / 2)
# val_size

# test_size = val_size
# test_size

# partition={'train':l[:train_size],
#            'val':l[train_size:train_size+val_size],
#            'test':l[train_size+val_size:train_size+val_size+test_size]}

# with open('nnet6_partition.pickle', 'wb') as handle:
#     pickle.dump(partition, handle, protocol=pickle.HIGHEST_PROTOCOL)

Loading `partition` dictionary

In [2]:
with open('nnet6_partition.pickle', 'rb') as handle:
    partition=pickle.load(handle)

The saved dictionary is 90MB as shown below:

In [13]:
!ls -lh nnet6_partition.pickle

-rw-rw-r--. 1 jayathu jayathu 91M Sep 10 12:30 nnet6_partition.pickle


In [3]:
params = {'dim': (16,64),
          'batch_size': 64,
          'n_classes': 5,
          'n_channels': 1,
          'shuffle': True}

In [4]:
# Generators
training_generator = DataGenerator2(partition['train'], [], **params)
validation_generator = DataGenerator2(partition['val'], [], **params)
testing_generator = DataGenerator2(partition['test'], [], **params)

# Model fitting

In [4]:
model = Sequential()
model.add(Flatten(input_shape=(*params['dim'],1)))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(5, activation='softmax'))

In [6]:
# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
filepath="models/nnet6/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=4,
                    epochs=5,
                    callbacks=callbacks_list,
                    verbose=1)

Epoch 1/5
100781/100783 [============================>.] - ETA: 0s - loss: 0.2708 - acc: 0.8791
Epoch 00001: val_acc improved from -inf to 0.91084, saving model to models/nnet6/weights-improvement-01-0.91.hdf5
100783/100783 [==============================] - 7412s 74ms/step - loss: 0.2708 - acc: 0.8791 - val_loss: 0.1918 - val_acc: 0.9108
Epoch 2/5
100782/100783 [============================>.] - ETA: 0s - loss: 0.1944 - acc: 0.9142
Epoch 00002: val_acc improved from 0.91084 to 0.92354, saving model to models/nnet6/weights-improvement-02-0.92.hdf5
100783/100783 [==============================] - 7398s 73ms/step - loss: 0.1944 - acc: 0.9142 - val_loss: 0.1712 - val_acc: 0.9235
Epoch 3/5
 23379/100783 [=====>........................] - ETA: 1:18:24 - loss: 0.1822 - acc: 0.9201

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 53731/100783 [==============>...............] - ETA: 47:40 - loss: 0.1812 - acc: 0.9207
Epoch 00003: val_acc improved from 0.92354 to 0.93116, saving model to models/nnet6/weights-improvement-03-0.93.hdf5
100783/100783 [==============================] - 7434s 74ms/step - loss: 0.1788 - acc: 0.9220 - val_loss: 0.1632 - val_acc: 0.9312
Epoch 4/5
 78517/100783 [======================>.......] - ETA: 22:39 - loss: 0.1706 - acc: 0.9257

# Saving model

In [7]:
# serialize model to JSON
model_json = model.to_json()
with open("models/nnet6/model.json", "w") as json_file:
    json_file.write(model_json)

# Evaluate model with test data

In [5]:
json_file = open('models/nnet6/model.json', 'r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights('models/nnet6/weights-improvement-03-0.93.hdf5')
# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Check if model is alright.

In [6]:
# evaluate the model
scores=model.evaluate_generator(testing_generator, steps=None, max_queue_size=10, workers=4, use_multiprocessing=1, verbose=1)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

21596/21596 [==============================] - 1311s 61ms/step

acc: 93.10%
